In [ ]:
%matplotlib inline

In [ ]:
import matplotlib
import numpy as np

In [ ]:
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rcParams['figure.figsize'] = (8, 4)   # Change the size of plots
matplotlib.rcParams['figure.dpi'] = 108

In [ ]:
def sgfit(x, y, va=None):
    if va is None:
        # Normal Gaussian fitting
        sum_x = np.sum(x)
        sum_x2 = np.sum(x ** 2)
        sum_x3 = np.sum(x ** 3)
        sum_x4 = np.sum(x ** 4)
        M = np.matrix([[len(x), sum_x, sum_x2], [sum_x, sum_x2, sum_x3], [sum_x2, sum_x3, sum_x4]])
        B = np.matrix([[np.sum(np.log(y))], [np.sum(np.multiply(x, np.log(y)))], [np.sum(np.multiply(x ** 2, np.log(y)))]])
        d = np.linalg.solve(M, B)
        a = d.item(0)
        b = d.item(1)
        c = d.item(2)

        A = np.exp(a - b ** 2 / (4 * c))
        mu = -b / (2 * c)
        if c > 0:
            # Failed
            sig = 1.0e-7
        else:
            sig = np.sqrt(-1 / (2 * c))
    else:
        # Normalize the x-values to [-pi, pi]
        omega = x / va * np.pi
        
        # Complex-plane representation
        c = np.multiply([max(0, x) for x in y], np.exp(1j * omega))
        
        # Angle of the sum of the vectors
        mu = np.angle(np.sum(c))
        
        # Shift the x-axis
        d = c * np.exp(-1j * mu)
        
        # New x-axis
        x = np.angle(d) / np.pi * va;
        
        sum_x2 = np.sum(np.abs(x) ** 2)
        sum_x4 = np.sum(np.abs(x) ** 4)
        a = len(x); b = c = sum_x2; d = sum_x4
        M = 1.0 / (a * d - b * c) * np.array([[d, -b], [-c, a]])
        B = np.array([np.sum(np.log(y)), np.sum(np.multiply(x ** 2, np.log(y)))])
        d = np.matmul(M, B)

        A = np.exp(d[0])
        mu = mu / np.pi * va
        sig = np.sqrt(-1 / (2 * d[1]))
    return A, sig, mu

In [ ]:
def spec(x, va, A, mu, sig, N):
    y = (A * np.exp(-(x - mu) ** 2 / (2 * sig ** 2))
    + A * np.exp(-(x - mu + 2 * va) ** 2 / (2 * sig ** 2))
    + A * np.exp(-(x - mu - 2 * va) ** 2 / (2 * sig ** 2))
    + 0.5 * N)
    return y

In [ ]:
N = 60
va = 15
x = np.arange(0, N) / N * 2 * va - va

A = 0.5
mu = 7
sig = 3

An = 1.0e-1 * A
# An = 0

y = spec(x, va, A, mu, sig, An)

n = An * (np.random.rand(N) - 0.5)
# n = 0

# Store a copy of the original
yo = y

# Add noise
y = y + n

# Some threshold to select what data samples to use
th = 0.5 * np.sqrt(np.mean(y ** 2))

# Mask of good samples
mask = y > th

# Say, we can offset the spectrum because we have a good noise estimate
y = y - 0.5 * An
y = np.multiply((1.1 * np.random.rand(N) + 0.5), y)

In [ ]:
A1, sig1, mu1 = sgfit(x[mask], y[mask])
y1s = spec(x, va, A1, mu1, sig1, An)

In [ ]:
A2, sig2, mu2 = sgfit(x[mask], y[mask], va)
y2s = spec(x, va, A2, mu2, sig2, An)

In [ ]:
highlight = [0.85, 0.96, 0]

tstr = 'mu = {0:.4f}   sig = {1:.4f}   A = {2:.4f} (ground truth)'.format(mu, sig, A)
m1 = 'mu = {0:.4f}   sig = {1:.4f}   A = {2:.4f} (conventional)'.format(mu1, sig1, A1)
m2 = 'mu = {0:.4f}   sig = {1:.4f}   A = {2:.4f} (SGFIT)'.format(mu2, sig2, A2)

fig = matplotlib.pyplot.figure()
fig.suptitle('Conventional Gaussing Fitting vs SGFIT', fontweight='bold')
if mask[0]:
    e = list(mask).index(False)
    matplotlib.pyplot.plot(x[0:e], y[0:e], color=highlight, linewidth=5)
    e = N - list(mask[::-1]).index(False)
    matplotlib.pyplot.plot(x[e:], y[e:], color=highlight, linewidth=5)
else:
    matplotlib.pyplot.plot(x[mask], y[mask], color=highlight, linewidth=5)
h1, = matplotlib.pyplot.plot(x[mask], y[mask], '.', color=highlight, markersize=20)
h2, = matplotlib.pyplot.plot(x, y, '.', MarkerSize=10, color=[0.85, 0.2, 0])
h3, = matplotlib.pyplot.plot(x, yo, color=[0.9, 0.6, 0])
h4, = matplotlib.pyplot.plot(x, y1s, '--', color=[0.7, 0.2, 0.8])
h5, = matplotlib.pyplot.plot(x, y2s, '-.', color=[0.4, 0.8, 0.0])
matplotlib.pyplot.grid()
matplotlib.pyplot.ylim((0.0, 2.0 * A))
matplotlib.pyplot.xlabel('Velocity (m/s)')
matplotlib.pyplot.ylabel('Amplitude')
matplotlib.pyplot.legend([h1, h2, h3, h4, h5], ['Used Samples', 'Original Dataset', tstr, m1, m2])
